**Make sure to have a copy of the database file, "world-db", from the last lecture downloaded and in this directory for the below to work!**

In [1]:
%load_ext sql
%sql sqlite:///./world-db

u'Connected: @./world-db'

The database `world` has 3 tables: 
> * `City`
> * `Country`
> * `CountryLanguage`

Recall that to find the schema of a table, we can type the following command:

In [2]:
%sql PRAGMA table_info(City);

 * sqlite:///./world-db
Done.


cid,name,type,notnull,dflt_value,pk
0,ID,int(11),1,None,1
1,Name,char(35),1,'',0
2,CountryCode,char(3),1,'',0
3,District,char(20),1,'',0
4,Population,int(11),1,'0',0


SINGLE-TABLE QUERIES
-------------------

Let's start with a simple query: what is the population of the USA?

In [3]:
%%sql 
SELECT Population 
FROM Country 
WHERE Code = 'USA';

 * sqlite:///./world-db
Done.


Population
278357000


**Exercise #1**: write a query that returns the countries that gained independence after 1989, along with the year they gained independence.

In [8]:
%%sql
SELECT name, IndepYear
FROM Country
WHERE Country.IndepYear > '1989'

 * sqlite:///./world-db
Done.


Name,IndepYear
Armenia,1991
Azerbaijan,1991
Bosnia and Herzegovina,1992
Belarus,1991
Czech Republic,1993
Eritrea,1993
Estonia,1991
"Micronesia, Federated States of",1990
Georgia,1991
Croatia,1991


To return all attributes of a table, we use the `*` in the SELECT clause. For instance, the following query returns all the attributes for cities with population over 1 million in the US.

In [7]:
%%sql 
SELECT *
FROM City
WHERE Population >= '1000000' AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


ID,Name,CountryCode,District,Population
3793,New York,USA,New York,8008278
3794,Los Angeles,USA,California,3694820
3795,Chicago,USA,Illinois,2896016
3796,Houston,USA,Texas,1953631
3797,Philadelphia,USA,Pennsylvania,1517550
3798,Phoenix,USA,Arizona,1321045
3799,San Diego,USA,California,1223400
3800,Dallas,USA,Texas,1188580
3801,San Antonio,USA,Texas,1144646


Keep in mind that for constants we need to use single quotes!

We can rename an attribute in the output schema by using the SQL command AS. For example:

In [9]:
%%sql
SELECT Name LargeUSACity
FROM City
WHERE Population >= '1000000' AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


LargeUSACity
New York
Los Angeles
Chicago
Houston
Philadelphia
Phoenix
San Diego
Dallas
San Antonio


It is possible to also perform arithmetic operations inside the SELECT clause. For example, to return the population in millions, we can use division.

In [10]:
%%sql
SELECT Name, ((Population / 1000000)+10) AS PopulationInMillion
FROM City
WHERE Population >= '2000000' AND CountryCode = 'USA';

 * sqlite:///./world-db
Done.


Name,PopulationInMillion
New York,18
Los Angeles,13
Chicago,12


Observe that division here behaves as integer division. If we want to get a float value, we can rewrite the above query (similar to type casting):

In [ ]:
%%sql
SELECT Name, ((Population * 1.0)/ 1000000) AS PopulationInMillion
FROM City
WHERE Population >= '2000000' AND CountryCode = 'USA';

Let's do some pattern matching! Find the countries that have a form of goverment that contains the word monarchy

In [ ]:
%%sql
SELECT Name, GovernmentForm
FROM Country
WHERE GovernmentForm LIKE '%Monarchy%';

Next, suppose that we want to find out all the different forms a government. Let's try the following:

In [ ]:
%%sql
SELECT GovernmentForm
FROM Country ;

The above query did not return the desired result, because it keeps a separate copy for each occurrence of the same value in the table. To remove the duplicates, we simply need to add the DISTINCT keyword.

In [ ]:
%%sql
SELECT DISTINCT GovernmentForm
FROM Country ;

We can use ORDER BY to order the city population by decreasing population.

In [ ]:
%%sql
SELECT Name, (Population/1000000) AS Pop
FROM City
WHERE Population >= '1000000'AND CountryCode = 'USA'
ORDER BY Name,Pop asc;

LIMIT is very useful if we want to look at only a few rows of a table.

In [ ]:
%%sql
SELECT *
FROM City
LIMIT 3 
order by Population;

**Exercise #2**: write a query that returns the two most populous cities in the US.

MULTI-TABLE QUERIES
---------------------

What are the names of all countries that speak Greek? To answer this question, we need to combine information from two tables by *joining* them.

In [ ]:
%%sql
SELECT Country.Name
FROM Country, CountryLanguage
WHERE Country.Code = CountryLanguage.CountryCode
AND CountryLanguage.Language = 'Greek';

Notice how we need to specify the table that each attribute comes from. Another way to write the above query:

In [ ]:
%%sql
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'Greek';

**Exercise #3**: write a query that returns the countries that speak at least 50% Greek.

**Exercise #4**: write a query that returns the district of the capital of USA.

Next, we want to return the countries that speak both Greek and English.

In [ ]:
%%sql
SELECT C.Name
FROM Country C, CountryLanguage L1, CountryLanguage L2
WHERE C.Code = L1.CountryCode
AND C.Code = L2.CountryCode
AND L1.Language = 'Greek'
AND L2.Language = 'English';

Alternatively, we can write the above query using INTERSECT.

In [ ]:
%%sql
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'Greek'
INTERSECT
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'English' ;

Which countries have population more than 100 million and do not speak English?

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Population >= 100000000
EXCEPT
SELECT C.Name
FROM Country C, CountryLanguage L
WHERE C.Code = L.CountryCode
AND L.Language = 'English' ;